In [1]:
# Dependencies
import tweepy
import json
import numpy as np
from config import consumer_key, consumer_secret, access_token, access_token_secret

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Target Search Term
target_terms = ("@BBCNews", "@CBSNews", "@CNN",
                "@FoxNews", "@nytimes")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# Array to hold sentiment
sentiment_array = []

# Variable for holding the oldest tweet
oldest_tweet = ""

# Loop through all target users
for target in target_terms:

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times (total of 1500 tweets)
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(target, count=100, result_type="recent")

        # Loop through all tweets
        for tweet in public_tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers and
                tweet["user"]["statuses_count"] > min_tweets and
                tweet["user"]["statuses_count"] < max_tweets and
                tweet["user"]["friends_count"] < max_following and
                tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                compound = analyzer.polarity_scores(tweet["text"])["compound"]
                pos = analyzer.polarity_scores(tweet["text"])["pos"]
                neu = analyzer.polarity_scores(tweet["text"])["neu"]
                neg = analyzer.polarity_scores(tweet["text"])["neg"]

                # Add each value to the appropriate array
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)

    # Store the Average Sentiments
    sentiment = {"User": target,
                 "Compound": np.mean(compound_list),
                 "Positive": np.mean(positive_list),
                 "Neutral": np.mean(negative_list),
                 "Negative": np.mean(neutral_list),
                 "Tweet Count": len(compound_list)}

    # Print the Sentiments
    print(sentiment)
    print("")


{'User': '@BBCNews', 'Compound': -0.08528918918918918, 'Positive': 0.05832432432432432, 'Neutral': 0.0995135135135135, 'Negative': 0.8421351351351353, 'Tweet Count': 370}

{'User': '@CBSNews', 'Compound': -0.013116331096196861, 'Positive': 0.09834451901565995, 'Neutral': 0.10393736017897091, 'Negative': 0.7977628635346756, 'Tweet Count': 447}

{'User': '@CNN', 'Compound': -0.0878687089715536, 'Positive': 0.07634354485776804, 'Neutral': 0.08191466083150985, 'Negative': 0.8416958424507659, 'Tweet Count': 457}

{'User': '@FoxNews', 'Compound': 0.12092973523421588, 'Positive': 0.12133808553971487, 'Neutral': 0.06214256619144603, 'Negative': 0.8165580448065172, 'Tweet Count': 491}

{'User': '@nytimes', 'Compound': -0.06148563535911603, 'Positive': 0.07862983425414365, 'Neutral': 0.11012983425414366, 'Negative': 0.8112679558011051, 'Tweet Count': 362}

